In [ ]:
import papermill
import targeter

In [ ]:
tar_pickle_path = './targeter.pickle'

In [ ]:
import pickle
file = open(tar_pickle_path, 'rb')
tar = pickle.load(file)
file.close()

\newpage

## Introduction

This report contains a snapshot of the specific subpopulation **«`r label(object$target,metadata)` » == `r object$target_reference_level``**. 

This subpopulation is characterized by  performing bivariate analyses, crossing the variable of interest (target) with  potential explanatory variables. Only most relevant (highest discriminative power) may be retained.

Report contains:

*   A summary overview table with selected retained criteria with some available information such as one statistic called information value (IV) that provides an idea of the explanatory power of the criteria.
*   Detailed analyses per criteria with graphics that highlight the reation between one of the candidate criteria and the _target_.

It can happen that one criteria contains some missing values. If such, we will introduce value *"[Missing]"* in the report, eventually with a cross **[x]**. Also, if two few data are available, we would use a minus **[-]** instead of reporting value. 

### Summary table interpretation

Criteria are sorted by decreasing importance order, which one is determined by the IV. 
One modality (value) index is also computed as the proportion of target «`r label(object$target,metadata)`» within the modality, that is the target penetration in the subpopulation selected by the specific modality of a criteria. A value near 1 would indicate a non-discriminative modality: selecting the corresponding subpopulation would result in the overall average target penetration. On the other way, the hightest the index (>1), the more target is over-representated in the subpopulation. Low index (<1) would be interpreted as under-representation. So high index modalities are of interest as they characterize subpopulation with high target penetration.

Note that index will always be computed for segments (modalities): continuous variables are cut into buckets. If a criteria/variable has no segment with any high index, it will not be of interest at all. Also, having one segment with an index  greater than 1 implies that there are counterparts segments with index lower 1.

There is no special rule to say that an index is a __good__ one as its interpretation has to take into account the natural (average) target pourcentage (penetration). Indeed, when the penetration is low, it is easier to observe eventually high index. Think that for a global penetration of 50% of target, maximal index is 2 (and more generally, index is bounded by `1/(target penetration)`)

For this report, we selected a maximum of  `r ntop` potential criteria.

For every criteria, summary table presents its possible value (modality) having the highest index along with the corresponding number of records. Pay attention that you might observe high index dealing with very restrictive sub-populations.

## Population description



## Target description
Target analysis :

In [ ]:
tar.target_stats

## Summary view


In [ ]:
tar.summary()

## Variables


In [ ]:
def format_df(df):
    """format the dataframe for display without index"""
    return display(HTML(df.to_html(index=False)))

# loop over variables
for ivar in list(tar.profiles.variable_names):
  # display title+anchor for links
  from IPython.display import display, Markdown
  display(Markdown(f"""
  ### {ivar} 
  
  """))
  vars =['Bin','Count', 'Count (%)', 'Event rate','WoE']
  format_df(tar.get_table(ivar)[vars])
  # plot
  tar.plot(ivar)
